<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install easyocr gradio pythainlp langchain langchain_huggingface langchain_community pytesseract transformers
!sudo apt-get install ghostscript

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


## **Job Qualification**

| **ᴊᴏʙ** | **ᴘᴏꜱɪᴛɪᴏɴ** | **ᴄᴏᴜɴᴛ** |
|:---:|:---:|:---:|
| 1 | ʙᴜꜱɪɴᴇꜱꜱ ᴀɴᴀʟʏꜱᴛ | 5 |
| 2 | ᴀɪ ᴇɴɢɪɴᴇᴇʀ | 7 |


In [7]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient
import pytesseract
from nltk.tokenize import sent_tokenize

In [33]:
resume_prompt = """
YOU ARE A HIGHLY SELECTIVE CAREER COACH AND RESUME EVALUATOR, TASKED WITH REVIEWING A CANDIDATE'S RESUME BASED ON A SPECIFIC JOB DESCRIPTION. DUE TO LIMITED RESOURCES AND SALARY CONSTRAINTS, YOU MUST PRIORITIZE ONLY THE MOST QUALIFIED CANDIDATES. YOUR ROLE IS TO STRICTLY ASSESS WHETHER THE RESUME MEETS OR EXCEEDS THE REQUIREMENTS OF THE JOB DESCRIPTION **PROVIDED IN THIS PROMPT**—EVEN IF THE CANDIDATE'S BACKGROUND IS IN A DIFFERENT FIELD.

### INSTRUCTIONS ###
- **ONLY USE THE JOB DESCRIPTION PROVIDED** to evaluate the resume. Do not rely on irrelevant qualifications or unrelated fields.
- PAY CLOSE ATTENTION to how the candidate's experience and skills match the specific requirements of the job description. Any significant deviation should be noted.
- CLASSIFY THE RESUME BASED ON HOW WELL IT ALIGNS WITH THE JOB REQUIREMENTS, using one of four categories: "PASS," "RECONSIDER," or "UNRELATED."

#### **Classification Criteria**:
1. **PASS**: The resume well aligns with the job description. The candidate meets or exceeds all key qualifications and has relevant experience that matches the specific job requirements.
2. **RECONSIDER**: The resume shows some alignment with the job requirements but has notable gaps or lacks some qualifications. The candidate may be worth considering further, potentially through an interview, to explore their fit for the role more deeply.
3. **UNRELATED**: The resume does not relate to the job description in anyway. The candidate's experience and qualifications are not applicable to the position role.

#### **CHAIN OF THOUGHT** (Reasoning Process):
1. **Step 1**: Review the **Job Description** and extract all the required and preferred qualifications.
2. **Step 2**: Compare the candidate’s **Resume** to the job requirements:
   - Identify relevant qualifications, skills, and experience from the resume.
   - **Note any irrelevant experience** that does not match the job description but do not immediately disqualify based on this alone.
3. **Step 3**: Classify the resume based on how closely it matches the job description. If the candidate’s background is from a different field but shows potential, consider the "RECONSIDER" category.
4. **Step 4**: Provide a clear and concise rationale for your classification, focusing on how well the resume aligns with the provided job description and any potential or gaps.

### **What Not To Do:**
- DO NOT AUTOMATICALLY DISQUALIFY A CANDIDATE IF THEY DO NOT MEET EVERY SINGLE REQUIREMENT. Consider their **potential** to adapt and grow into the role based on their existing skills and experience.
- AVOID STRICTLY PENALIZING CANDIDATES FOR **MISMATCHES**. Take into account their **transferable skills** and ability to contribute in ways that might not be immediately obvious.
- DO NOT IGNORE THE PROVIDED JOB DESCRIPTION—IT IS THE PRIMARY BASIS FOR CLASSIFICATION, BUT REMAIN OPEN TO CANDIDATES WHO SHOW PROMISE IN OTHER AREAS.

### **Job Description: (never copy it)**
{jobdescribe}

### **Resume: (never copy it)**
{resume}

### **Evaluation** (show only this part):###
- **Classification**: (Pass / Reconsider / Unrelated)
- **Rationale**: Provide a detailed justification for your classification, focusing on how well the candidate aligns with both the required and preferred qualifications. Highlight any exceptional strengths or critical gaps in the resume.

### **Few-Shot Example (never copy it)**:
### **Job Description**:
- Required: 5+ years in software development, expertise in Python and JavaScript, strong familiarity with cloud platforms, advanced leadership or team management skills, and a bachelor's degree in Computer Science or a related field. Preferred: Master’s degree or higher, experience with DevOps, and published technical papers.

### **Resume**:
- Candidate has 6 years of experience in software development, proficient in Python and JavaScript, has worked with cloud platforms (AWS), and has strong leadership experience leading teams of up to 10 people. Holds a Master’s degree in Computer Science and has experience in DevOps.

### **Evaluation**:
- **Classification**: Pass
- **Rationale**: The candidate exceeds the job requirements by having both the required skills (Python, JavaScript, cloud platforms, leadership) and the preferred qualifications (Master’s degree, DevOps experience). They demonstrate strong alignment with both the required and preferred criteria.

### **Example Evaluation with Transferable Skills** (never copy it):
### **Job Description**:
- Required: 3+ years of experience in software development, proficiency in Python and JavaScript, experience with cloud computing, and a bachelor’s degree in Computer Science.
- Preferred: Experience with React.js, AWS, and knowledge of DevOps practices.

### **Resume**:
- The candidate has over 3 years of experience in data analysis and automation, proficient in Python, and has worked on several projects related to cloud platforms like AWS. They have experience in process automation, but less direct experience with JavaScript and DevOps.

### **Evaluation**:
- **Classification**: Reconsider
- **Rationale**: The candidate shows promise with strong Python and cloud computing skills and experience in process automation. Although there are gaps in JavaScript and DevOps, their background suggests potential value. Further discussion or an interview might clarify their fit for the role.

### **Example Evaluation with Mismatched Job Description and Resume (never copy it):**
### **Job Description**:
- Required: 3+ years of experience in software development, proficiency in Python and JavaScript, experience with cloud computing, and a bachelor’s degree in Computer Science.
- Preferred: Experience with React.js, AWS, and knowledge of DevOps practices.

### **Resume**:
- The candidate has over 2 years of working experience as a content creator in the marketing industry.
- The candidate has knowledge of using various media platforms and basic graphic design and editing skills.
- The candidate holds a Bachelor of Business Administration (BBA) degree.
- The candidate has experience in digital marketing, graphic design for online and offline media, and coordinating with partners and customers.
- The candidate can use various graphic design and video editing software, such as Photoshop, Microsoft Office, Canva, Inshot, and has experience with quality control for social media pages.

### **Evaluation**:
- **Classification**: Unrelated
- **Rationale**: The candidate’s background in content creation and marketing does not align with the technical requirements of the software development role. Their experience and qualifications are not applicable to the position.
"""

In [35]:
# Define the model parameters
model_params = {
    "max_new_tokens": 1500,
    "temperature": 0.1,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}
semaphore = Semaphore(50)

def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",
            f"-r{dpi}",
            "-o", output_format,
            pdf_file
        ]
        subprocess.run(gs_command, check=True)
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

def chunk_text(text, tokenizer, max_tokens=200):
    tokens = tokenizer.tokenize(text)
    total_tokens = len(tokens)
    chunks = []
    start = 0
    while start < total_tokens:
        end = min(start + max_tokens, total_tokens)
        chunk = tokenizer.convert_tokens_to_string(tokens[start:end])
        chunks.append(chunk)

        start += max_tokens

    return chunks

def tag_and_clean_text(text, tagger, tokenizer, unwanted_pattern, max_tokens=200):
    text_chunks = chunk_text(text, tokenizer, max_tokens=max_tokens)
    tagged_text = []
    cleaned_text = []

    for i, chunk in enumerate(text_chunks):
        ner = tagger.get_ner(chunk, tag=True)
        if not ner:
            ner = chunk

        # Clean the tags
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]*?</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        # Append the tagged and cleaned text
        tagged_text.append(ner)
        cleaned_text.append(cleaned_ner)
    # Combine results from all chunks
    combined_tagged_text = "\n".join(tagged_text).strip()
    combined_cleaned_text = "\n".join(cleaned_text).strip()

    return combined_tagged_text, combined_cleaned_text


# Function to generate an answer using the prompt
def generate_answer(resume, job_description):
    formatted_prompt = resume_prompt.replace("{resume}", resume).replace("{jobdescribe}", job_description)
    truncated_prompt = formatted_prompt[:model_params["max_new_tokens"]]
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Extract classification from the result text
def extract_classification(text):
    pattern = r'\*\*\s*Classification\s*\**:\s*(?:\[)?\s*(Pass|Reconsider|Not Pass|Unrelated)\s*(?:\])?'
    match = re.search(pattern, text, re.IGNORECASE)
    if not match:
        pattern = r'Classification\s*:\s*(?:\[)?\s*(Pass|Reconsider|Not Pass|Unrelated)\s*(?:\])?'
        match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    return text

def process_file(file, job_description, is_pdf=True):
    if is_pdf:
        images = convert_pdf_to_images(file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"
    else:
        raw_text = extract_text_from_image(file.name)

    tagger = NamedEntityTagger()
    tokenizer = tagger.tokenizer

    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me', 'birth', 'date',
        'address', 'email.', 'ประวัติ'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer, unwanted_pattern)
    evaluation = generate_answer(cleaned_text, job_description)
    result = extract_classification(evaluation)

    df = pd.DataFrame([{
        "File": os.path.basename(file.name),
        "Raw_Text": raw_text,
        "Tagged_Text": tagged_text,
        "Cleaned_Text": cleaned_text,
        "Job_Description": job_description,
        "Evaluation": evaluation,
        "Result": result
    }])
    return df

# Function to process multiple files
def process_multiple_files(files, job_description, is_pdf=True):
    all_results = pd.DataFrame()
    for file in files:
        df = process_file(file, job_description, is_pdf)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Function to save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Function to process a batch of DataFrame rows
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    job_description = row.get('Job_Description', "")
    result = generate_answer(resume, job_description)
    return result

# Function to process a batch of rows
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

# Gradio Interface Logic
def gradio_interface(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
    try:
        # Update model parameters
        model_params.update({
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty
        })

        # Process files and pass job description
        df = process_multiple_files(files, job_description, is_pdf)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Result'] = df['Evaluation'].apply(extract_classification)

        # Select only the required columns for output
        df_output = df[["Result", "Evaluation"]]

        csv_path = save_dataframe(df)
        return df_output, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Define the Gradio interface
def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Row():
            # Left Column for File Input
            with gr.Column(scale=1):
                file_input = gr.Files(label="Upload Files", file_count="multiple")
                is_pdf_input = gr.Checkbox(label="PDF", value=True)

            # Right Column for Parameters and Job Description
            with gr.Column(scale=1):
                job_input = gr.Textbox(placeholder="Enter the job description here...", label="Job Description", lines=10)

                # Model parameters
                max_new_tokens_input = gr.Slider(label="Max New Tokens", minimum=50, maximum=1024, value=200, step=50)
                temperature_input = gr.Slider(label="Temperature", minimum=0.01, maximum=1.0, value=0.1, step=0.1)
                top_p_input = gr.Slider(label="Top P", minimum=0.1, maximum=1.0, value=0.95, step=0.05)
                repetition_penalty_input = gr.Slider(label="Repetition Penalty", minimum=0.1, maximum=2.0, value=1.0, step=0.1)

                submit_btn = gr.Button("Process")

        # Center Bottom for Output
        with gr.Row():
            with gr.Column(scale=2):
                # Output DataFrame with limited rows visible and interactivity off
                output_df = gr.DataFrame(
                    headers=["Result", "Evaluation"],
                    type="pandas",
                    interactive=False
                )

                output_csv = gr.File(label="Download CSV")

        # Function to process inputs and generate output
        def on_submit(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
            df, csv_path = gradio_interface(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty)
            return df, csv_path

        # Bind the submit button to the processing function
        submit_btn.click(on_submit,
                         inputs=[file_input, job_input, is_pdf_input, max_new_tokens_input, temperature_input, top_p_input, repetition_penalty_input],
                         outputs=[output_df, output_csv])

    return demo

In [36]:
app = create_gradio_interface()
app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://381090bcd0b972410a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Error in gradio_interface: 'NoneType' object is not iterable


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1945, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1770, in postprocess_data
    outputs_cached = await processing_utils.async_move_files_to_cache(
  File "/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py", line 485, in async_move_files_to_cache
    return await client_utils.async_traverse(
  File "/usr/local/lib/python3.10/dist-packages/gradio_client/utils.py", line 999, in async_traverse
    return await func(json_obj)
  File "/usr/local/lib/python3.

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://381090bcd0b972410a.gradio.live


how the `chunking process` handles boundaries between tokens when the text is split, leading to some parts of the text getting lost, especially near the chunk boundaries. This often occurs when chunks are cut off abruptly, and during the reassembly after tagging, the missing parts may not get included properly.

In [28]:
import pandas as pd
a = pd.read_csv('/content/output.csv')
a

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
0,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated
1,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Rec...,Reconsider
2,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ การศึกษา มหาวิทยาลัยเชียงใ...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated
3,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated
4,resume ai4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณํ ทํางาน วิศวกร a...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated
5,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated
6,resume ai6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,<URL>linkedin</URL>:<PERSON> </PERSON><PERSON>...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated
7,resume ai7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Rec...,Reconsider
8,resume ai8.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Rec...,Reconsider
9,Resume ba1.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,Minimum qualifications:\nBachelor's in Enginee...,### **Evaluation**:\n- **Classification**: Unr...,Unrelated


In [32]:
print(a['Evaluation'].iloc[0])

### **Evaluation**:
- **Classification**: Unrelated
- **Rationale**: The candidate's experience and skills do not align with the job requirements. The resume highlights software engineering and development skills, particularly in Java, Python, and related technologies, but does not demonstrate experience in designing and implementing control systems, controls modeling, or Machine Learning as required by the job description. Additionally, the resume does not show relevant experience with programming languages such as C, MATLAB, or Shell, which are specifically mentioned in the job description. The candidate's background in software engineering and development, while valuable, does not meet the specific technical qualifications needed for this role. There is no evidence of experience in data center efficiency optimization, control algorithms for electrical and mechanical stability, or statistical analysis/modeling relevant to data center controls. Therefore, the resume is unrelated to th

In [ ]:
## job = BA(https://th.jobsdb.com/Business-Analysis-jobs?jobId=78649722&type=standout) resume AI and BA
## job = BA(https://th.jobsdb.com/AI-Engineer-jobs?jobId=78442339&type=standout)) resume AI and BA

import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI.to_csv('/content/drive/MyDrive/AIEngineer/resume/JOB/TEST_AI+BA.csv')
AI

In [ ]:
## job = BA(https://th.jobsdb.com/Business-Analysis-jobs?jobId=78649722&type=standout) resume AI and BA
## job = BA(https://th.jobsdb.com/AI-Engineer-jobs?jobId=78442339&type=standout)) resume AI and BA

import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI.to_csv('/content/drive/MyDrive/AIEngineer/resume/JOB/TEST_AI+BA.csv')
AI

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
0,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
1,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ ประวัติการศึกษา มหาวิทยาลั...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
2,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Not...,Not Pass
3,resume ai4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณํ ทํางาน วิศวกร a...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
4,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
5,resume ai6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON> </PERSON><PERSON>น</PERSON><...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
6,resume ai7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
7,resume ai8.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
8,Resume ba1.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Not...,Not Pass
9,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทํางาน flnancial an...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Not...,Not Pass


In [ ]:
print(AI['Evaluation'].iloc[2])

### **Evaluation**:
- **Classification**: Not Pass
- **Rationale**: The candidate's resume does not align with the job description for the senior AI engineer position. The resume is written in Thai and appears to describe experience in AI and machine learning, but it does not provide clear evidence of the required qualifications. Specifically, the resume does not mention hands-on knowledge in AI, machine learning, deep learning, PyTorch, Python, NLP, or any of the specialized machine learning areas listed in the job description. The candidate's experience with scikit-learn and Keras is mentioned, but this does not necessarily meet the requirement for hands-on knowledge in the specified areas. Additionally, the resume does not provide evidence of proven experience in applying AI to practical technology solutions, knowledge of various deep learning model architectures, or experience with training deep learning models on large datasets. The candidate's programming skills in Python and C++

In [ ]:
print(BA['Evaluation'].iloc[11])

### **Evaluation**:
- **Classification**: Not Pass
- **Rationale**: The candidate's resume does not align with the job description for a business analyst role. The candidate's experience is primarily in business analysis within the IT sector, focusing on technology solutions and process improvements. However, the job description requires experience in market research, competitive analysis, and business modeling, which are not prominently featured in the resume. Additionally, the candidate's educational background in Economics does not directly match the preferred qualifications, which include Business Administration, Finance, or related fields. The resume lacks specific skills and experience in market analysis, financial modeling, and strategic planning that are critical for the role described. Therefore, the candidate does not meet the key requirements for the position.


In [ ]:
print(BA['Raw_Text'].iloc[11])

 จิตรลดา
086-234-5678
jitlada.s@domain. com
โสภาเสริม
321 ถนนอโศกมนตรี, กรุงเทพฯ 10120
linkedin .com/in /jitlada-s
 business analyst
เป้าหมายในการทำงาน
ต้องการนำทักษะด้านการวิเคราะห์และความรู้เชิงธุรกิจมาช่วยเพิ่มประสิทธิภาพและผลลัพธ์ในการดำเนินธุรกิจขององค์กร
ประสบการณ์การทำงาน
ปัจจุบัน
 บริษัท modern solutions
 business analyst
เมษายน 2019
 ทำงานร่วมกับทีมเทคโนโลยีสารสนเทศและผู้บริหารในการระบุและแก้ไขปัญหาทางธุรกิจ
วิเคราะห์ ข้อมูลลูกค้าและจัดทำรายงานเพื่อสนับสนุนการตัดสินใจ
 ปรับปรุงกระบวนการทำงานภายในองค์กรเพื่อเพิ่มประสิทธิภาพ
 junior business analyst
 บริษัท nextgen tech
มีนาคม 2019
 มกราคม 2016
.
รวบรวมและวิเคราะห์ข้อมูลเพื่อสนับสนุนการตัดสินใจของผู้บริหาร
ประสานงานกับทีมพัฒนาในการจัดทำและทดสอบระบบใหม่
การศึกษา
 ปริญญาตรี สาขาเศรษฐศาสตร์
มหาวิทยาลัยเชียงใหม่, เชียงใหม่
2011
2015
ทักษะ
การวิเคราะห์ธุรกิจ: brd, frd, gap analysis
การจัดการโครงการ: microsoft project, trello
การวิเคราะห์ข้อมูล: excel, power bi, sol
การสื่อสาร: การเขียนรายงาน, การนำเสนอ



In [ ]:
print(BA['Tagged_Text'].iloc[11])

<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON>ลด</PERSON><PERSON>า</PERSON><PHONE> 08</PHONE><PHONE>6-2</PHONE><PHONE>34-5678</PHONE><PERSON> </PERSON><PERSON>jit</PERSON><PERSON>lada</PERSON><EMAIL>.</EMAIL><EMAIL>s@domain. com</EMAIL><LOCATION> </LOCATION><PERSON>โสภ</PERSON><PERSON>า</PERSON>เสริม<LOCATION> 3</LOCATION><LOCATION>21</LOCATION><LOCATION> </LOCATION><LOCATION>ถนนอโศกมนตรี</LOCATION><LOCATION>,</LOCATION><LOCATION> </LOCATION><LOCATION>กรุงเทพฯ</LOCATION><ZIP> 10</ZIP><ZIP>120</ZIP><URL> linkedin .com/in /</URL><PERSON>jit</PERSON><PERSON>lada</PERSON><URL>-s</URL><ORGANIZATION> business</ORGANIZATION> analyst เป้าหมายในการทํางาน ต้องการนําทักษะด้านการวิเคราะห์และความรู้เชิงธุรกิจมาช่วยเพิ่มประสิทธิภาพและผลลัพธ์ในการดําเนินธุรกิจขององค์กร ประสบการณ์การทํางาน ปัจจุบัน<ORGANIZATION> บริษัท modern solutions business analyst</ORGANIZATION><DATE> </DATE><DATE>เมษายน 2019</DATE> ทํางานร่วมกับทีมเทคโนโลยีสารสนเทศและผู้บริหารในการระบุและแก้ไขปัญหาทางธุรกิจ วิเคราะห์ ข้อมูลลูกค

RULE
- AI ทั้งหมด ไม่ผ่าน แต่ BA ยังมี BORDERLINE
-

In [ ]:
print(AI['Evaluation'].iloc[2])

Job Description:
- Required: 4+ years of experience in AI development, proficiency in Python and C++, ability to handle large data sets, experience with machine learning frameworks (Scikit-learn, TensorFlow, Keras), knowledge of image processing (OpenCV, Pillow), and a bachelor’s degree in Computer Science or a related field.

Resume:
- The candidate has 4 years of experience in AI development, specifically in developing AI solutions for data analysis. They are proficient in Python and C++, and have experience with Scikit-learn and Keras. They have skills in handling large data sets and creating models that improve business decision-making. They have experience with image processing using OpenCV and Pillow. They have a background in machine learning and deep learning using TensorFlow and Keras. They have a current role as a data engineer, developing machine learning models for financial forecasting to improve strategic decision-making. They have collaborated with data analysis teams to

In [ ]:
import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,RESUME AI ENG.pdf,"narakorn vorakul\n18/4 pak nam, mmueang samut ...",<PERSON>nar</PERSON><PERSON>akorn vorakul</PER...,address email.v ai engineer ตําแหน่ง birth da...,Strengths: [List the candidate's strengths as ...,Borderline
1,RESUME AI TH.pdf,นางสาวนารากร วรกุล\nที่อยู่:\n18/4 ถ.ศรีสมุทร ...,<PERSON>น</PERSON><PERSON>นางสาว</PERSON><PERS...,ai engineer ตําแหน่ง แนะนําตัวเบื้องต้น ผู้ ส...,Skills: [List the key skills that the candidat...,Not Pass
2,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Additional Notes: [Provide any additional note...,Pass
3,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ ประวัติการศึกษา มหาวิทยาลั...,Qualifications: [Score / 10]\nSkills: [Score /...,Pass
4,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Strengths: [List at least two strong qualifica...,Pass
5,resume ai4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณํ ทํางาน วิศวกร a...,Skills & Experience: [Score out of 10]\nCultur...,Pass
6,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,Standout Qualifications: [List any qualificati...,Pass
7,resume ai6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON> </PERSON><PERSON>น</PERSON><...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...,Skills Alignment: [Assess the alignment of the...,Pass
8,resume ai7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...,Strengths: [List the candidate's strengths tha...,Pass
9,resume ai8.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...,Qualifications: [Evaluate the match between th...,Pass


In [ ]:
print(AI['Tagged_Text'].iloc[0])

<PERSON>nar</PERSON><PERSON>akorn vorakul</PERSON><LOCATION> 18</LOCATION><LOCATION>/4 pak nam</LOCATION><LOCATION>,</LOCATION><LOCATION> mmueang</LOCATION><LOCATION> samut prakan</LOCATION><LOCATION> district</LOCATION><LOCATION>,</LOCATION><LOCATION> samut prakan</LOCATION><ZIP> 10</ZIP><ZIP>270</ZIP> address:<PHONE> 09</PHONE><PHONE>9</PHONE><PHONE>-827-8887</PHONE> phone: email:<PERSON> nar</PERSON><PERSON>akorn</PERSON>.v<EMAIL>@</EMAIL><EMAIL>kkumail.com</EMAIL> al engineer ตําแหน่ง: birth date:<DATE> april 23, 2002</DATE> summary enthusiastic and detail oriented recent graduate with a bachelor's degree in statistics and data science, looking for an appropriate position. demonstrates strong analytical skills, attention to detail, and proficiency in python. experienced in data cleaning, preprocessing, machine learning implementation, and web application development with python., gained through academic projects and internships. education may bachelor of science (statistics and dat

In [ ]:
import pandas as pd
BA = pd.read_csv('/content/output.csv')
BA

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,Skills Assessment: \n- [Skill 1]: [Level of p...,Borderline
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...,Skills & Qualifications: [Assess the skills an...,Pass
2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,Job Description Requirements: [List the key re...,Pass
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทํางาน flnancial an...,### **Job Description:**\nWe are seeking an ex...,Pass
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,Specific Skill Set: [List the specific skills ...,Pass
